Name : Hazem Bin Ryaz Patel (2200550)
Class : DAAA/2B/07 


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import visualkeras
import keras
import numpy as np
from keras.layers import (
    AveragePooling2D,
    ZeroPadding2D,
    BatchNormalization,
    Activation,
    MaxPool2D,
    Add,
)
from keras.models import Sequential
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Normalization, Dense, Conv2D, Dropout, BatchNormalization, ReLU
from keras.models import Sequential
from keras.models import Model
from keras.optimizers import *
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.utils.vis_utils import plot_model
# from scikeras.wrappers import KerasClassifier, KerasRegressor
# from sklearn.model_selection import RandomizedSearchCV,KFold


In [ ]:
seed_r = 42
np.random.seed(seed_r)

# EDA

## Loading in data

In [ ]:
data_dir = "./Dataset for CA1 part A"
# image_count = len(list(data_dir.glob('*/*.jpg')))

batch_size = 32
img_height = 224
img_width = 224

train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "/train",
    seed=seed_r,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "/validation",
    seed=seed_r,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir + "/test",
    seed=seed_r,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

## Visualizing the data

In [ ]:
class_names = train_ds.class_names
print(len(class_names))

In [ ]:
x_train = []
y_train = []

for images, labels in train_ds:
    x_train.extend(images.numpy())
    y_train.extend(labels.numpy())

x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 5), tight_layout=True)

for label, subplot in enumerate(ax.ravel()):
    subplot.axis("off")
    subplot.imshow(
        x_train[y_train == label][
            np.random.randint(0, len(x_train[y_train == label]))
        ].astype("uint8"),
        cmap="Greys",
    )
    subplot.set_title(class_names[label])

plt.show()

## Checking for mislabelled data

In [ ]:
fig, ax = plt.subplots(15, 10, figsize=(15, 20))
for i in range(15):
    images = x_train[np.squeeze(y_train == i)].astype("uint8")
    random_index = np.random.choice(images.shape[0], 15, replace=False)
    images = images[random_index]
    label = class_names[i]
    for j in range(10):
        subplot = ax[i, j]
        subplot.axis("off")
        subplot.imshow(images[j], cmap='Greys')
        subplot.set_title(label, fontsize=8)

plt.show()

Conclusion : There isn't any mislabelling, so we don't need to re-label any of the data

## Image Averaging

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(20, 10))

for idx, subplot in enumerate(ax.ravel()):
    avg_image = np.mean(x_train[np.squeeze(y_train == idx)], axis=0) / 255
    subplot.imshow(avg_image, cmap="Greys")
    subplot.set_title(f"{class_names[idx]}")
    subplot.axis("off")

Some of them you can kinda see the color, but most of them is just a ball of green

## Distribution of Classes

In [ ]:
labels, counts = np.unique(y_train, return_counts=True)
for label, count in zip(labels, counts):
    print(f"{class_names[label]}: {count}")

plt.barh(labels, counts, tick_label=class_names)
plt.show()

# Pre-Processing 

## Oversampling with the use of Data Augmentations
One of the things that is very obvious that neeeds to be adjusted is the imabalance of classes
The method that I've chose to address this is oversampling of the classes which are imbalanced


Easy access to images and labels: You can directly access any image and its label using the tensor reference as the key.
Simplified code: The code is more straightforward and easier to understand.
Disadvantages:

Memory usage: Unbatching the dataset and storing it in a dictionary can consume more memory, especially for large datasets.
Slower execution: Iterating over the entire unbatched dataset to create the dictionary can be slower than iterating over a batched dataset.
Directly iterating over a batched dataset:

In [ ]:
train_dict = {tf.Tensor.ref(img): label for img, label in train_ds.unbatch()}

def data_augmentation(data):
    imageArr = []
    for images in data:
        image = tf.image.random_flip_left_right(images)
        image = tf.image.random_crop(
            image, size=(224,224,3)
        )
        imageArr.append(tf.reshape(image, (224, 224, 3)))
    return np.array(imageArr)

def augment_undersampled_vegs(img_labels, X_train, y_train):
    undersampled_labels = []
    undersampled_vegs = []
    for veg_type in img_labels:
        # Get all images of a veg type
        veg_images = [img.deref() for img, label in train_dict.items() if label == veg_type]
        veg_labels = [label for img, label in train_dict.items() if label == veg_type]

        if veg_type == img_labels[0]:
            undersampled_vegs = veg_images
            undersampled_labels = veg_labels     
        else : 
            undersampled_vegs = np.concatenate((undersampled_vegs, veg_images), axis=0)
            undersampled_labels = np.concatenate((undersampled_labels, veg_labels), axis=0)

        veg_train_aug = data_augmentation(undersampled_vegs)
    
    print(veg_train_aug.shape)
    print(undersampled_labels.shape)

    X_train = np.concatenate((X_train, veg_train_aug), axis=0)
    y_train = np.concatenate((y_train, undersampled_labels), axis=0)
    return X_train, y_train

# veg_types are the labels of the vegetables which are undersampled
veg_types = [2,5,6,7,10,11,13]

X_train_aug, y_train_aug = augment_undersampled_vegs(veg_types, x_train, y_train)

### Processing Images
- Re-batch X_train_aug and y_train_aug
- Grayscale, Normalize and Resize

In [ ]:
with tf.device('/device:CPU:0'):
    train_ds_rebatch = tf.data.Dataset.from_tensor_slices((X_train_aug, y_train_aug))
    train_ds_rebatch = train_ds_rebatch.shuffle(buffer_size=len(X_train_aug))  # Shuffle the data
    train_ds_rebatch = train_ds_rebatch.batch(32)

# normalization_layer = tf.keras.layers.Rescaling(1.0 / 255)
# ds = ds.map(lambda x, y: (normalization_layer(x), y))
def process(ds):
    ds = ds.map(lambda x, y: (tf.image.rgb_to_grayscale(x), y))
    ds = ds.map(lambda x, y: (tf.image.resize(x, (128, 128)), y))
    return ds

train_ds_128 = process(train_ds_rebatch)
val_ds_128 = process(val_ds)
test_ds_128 = process(test_ds)

## EDA After Resizing, Grayscaling & Normalization

In [ ]:
x_train_128 = []
y_train_128 = []

for images, labels in train_ds_128:
    x_train_128.extend(images.numpy())
    y_train_128.extend(labels.numpy())

x_train_128 = np.array(x_train_128)
y_train_128 = np.array(y_train_128)

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8,5), tight_layout=True)

for idx, subplot in enumerate(ax.ravel()):
    avg_image = np.mean(x_train_128[np.squeeze(y_train_128 == idx)], axis=0) / 255
    subplot.imshow(avg_image, cmap="Greys")
    subplot.set_title(f"{class_names[idx]}")
    subplot.axis("off")

In [ ]:
fig, ax = plt.subplots(3, 5, figsize=(8, 5), tight_layout=True)

for label, subplot in enumerate(ax.ravel()):
    subplot.axis("off")
    subplot.imshow(
        x_train_128[y_train_128 == label][
            np.random.randint(0, len(x_train_128[y_train_128 == label]))
        ],
        cmap="Greys",
    )
    subplot.set_title(class_names[label])

plt.show()

## Training for CNN 128


In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds_128 = train_ds_128.cache().prefetch(buffer_size=AUTOTUNE)
val_ds_128 = val_ds_128.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
def create_model_128():  # learning_rate, activation
    model = Sequential()
    model.add(
        Conv2D(64, 3, input_shape=(128, 128, 1), padding="same", activation="relu")
    )
    model.add(MaxPooling2D(2, 2))
    model.add(BatchNormalization())
    model.add(Conv2D(128, 3, padding="same", activation="relu"))

    model.add(MaxPooling2D(2, 2))
    model.add(Dropout(0.5))

    model.add(Conv2D(64, 3, padding="same", activation="relu"))
    model.add(MaxPooling2D(2, 2))
    model.add(Conv2D(32, 3, padding="same", activation="relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    # Flatten the feature map
    model.add(Flatten())

    # Add the fully connected layers
    model.add(Dense(512, activation="relu"))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(15, activation="softmax"))

    # Compile your model with your optimizer, loss, and metrics
    model.compile(
        optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"]
    )

    return model

model_128 = create_model_128()
history_128 = model_128.fit(
    train_ds_128, 
    validation_data=val_ds_128, 
    epochs=30, 
    batch_size=64,
    callbacks=[EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min'), 
    ModelCheckpoint('model_128.h5', monitor='val_loss', save_best_only=True, verbose=1)]
)

model.summary()
plot_model(model_128, show_shapes=True, show_layer_names=True)

In [ ]:
model_128.evaluate(test_ds_128)

In [ ]:
plt.plot(history_128.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# summarize history for loss
plt.plot(history_128.history['loss'])
plt.plot(history_128.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Hyperparameter Tuning for 128 model

In [ ]:
def tune_128_model(dropout, batchsize, dense):
    opt = Adam(lr=0.01)

    model = Sequential()

    model.add(Conv2D(32, (3, 3), input_shape=(31, 31, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Conv2D(32, (3, 3), input_shape=(31, 31, 1), activation='relu'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization())

    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(dropout))

    model.add(Flatten()) 
    model.add(Dense(128, activation='relu'))  # hidden layer
    model.add(Dense(dense, activation="softmax"))  # output layer

    # Compile your model with your optimizer, loss, and metrics
    model.compile(
        optimizer=opt,
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"]
    )

    history = model.fit(train_ds_128, validation_data=val_ds_128, epochs=30, batch_size=batchsize, shuffle=True)

    # Evaluate model on unseen data
    scores = model.evaluate(test_ds)
    testError = 100-scores[1]*100

    return history, scores[1], testError

In [ ]:
def grid_search(batch_sizes, dropouts, dense_sizes):
    # Store the results
    results = []
    # Iterate over all combinations of hyperparameters
    for batch_size in batch_sizes:
        for dropout in dropouts:
            for dense_size in dense_sizes:
                history, accuracy, test_error = tune_128_model(batch_size, dropout, dense_size)
                
                # Store the results
                results.append({
                    'batch_size': batch_size,
                    'dropout': dropout,
                    'dense_size': dense_size,
                    'accuracy': accuracy,
                    'test_error': test_error
                })

    return results

# Define the hyperparameters to test
batch_sizes = [32, 64, 128]
dropouts = np.arange(0.1,0.5,0.8)
dense_sizes = [32,64,128,256]

# Run the grid search
results = grid_search(batch_sizes, dropouts, dense_sizes)

## Load in model from .h5 file